In [9]:
import rawpy
import imageio.v3 as iio
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from pathlib import Path
load_dotenv()

# Charger le fichier RAF
base_path = os.getenv("BASE_PATH")
path = base_path / "DSCF3552.RAF"
with rawpy.imread(path) as raw:
    rgb = raw.postprocess()

# Afficher l’image
plt.figure(figsize=(10, 7))
plt.imshow(rgb)
plt.axis('off')
plt.show()

TypeError: unsupported operand type(s) for /: 'NoneType' and 'str'